In [12]:
import pandas as pd
import unidecode

# Leer el archivo CSV
df = pd.read_csv('INVENTOR6724.csv')

# Seleccionar las columnas necesarias
df['NOMBRE DEL INVESTIGADOR PATENTE'] = df['inventor_last_name'] + ',' + df['inventor_first_name']

# Seleccionar y formatear nombres
df['NOMBRE DEL INVESTIGADOR PATENTE'] = (
    df['inventor_last_name'] + ',' + df['inventor_first_name']
    ).str.replace(r'[-]', ' ', regex=True).str.replace(r'[.]', '', regex=True).apply(unidecode.unidecode).str.upper()

# Ordenar en orden alfabético
df = df.sort_values(by='NOMBRE DEL INVESTIGADOR PATENTE').drop_duplicates(subset=['NOMBRE DEL INVESTIGADOR PATENTE'])

# Guardar en un nuevo archivo CSV
df[['NOMBRE DEL INVESTIGADOR PATENTE']].to_csv('Nombres_Patentes.csv', index=False)

In [13]:
# Leer el archivo Nombres_Limpios_Final
df_nombres_limpios = pd.read_csv('Nombres_Limpios_Final.csv')

# Buscar coincidencias
df_match = df[df['NOMBRE DEL INVESTIGADOR PATENTE'].isin(df_nombres_limpios['NOMBRE DEL INVESTIGADOR'])]

# Guardar las coincidencias en un nuevo archivo CSV
df_match[['NOMBRE DEL INVESTIGADOR PATENTE']].rename(columns={'NOMBRE DEL INVESTIGADOR PATENTE': 'NOMBRE DEL INVESTIGADOR MATCH'}).to_csv('Match.csv', index=False)

In [7]:
import pandas as pd
from fuzzywuzzy import fuzz

# Cargar datasets
df_datasetF = pd.read_csv("datasetF.csv")
df_inventor = pd.read_csv("INVENTOR6724.csv")
df_match = pd.read_csv("Match_2_Limpio.csv")

# Normalización de nombres y ubicaciones
def normalize_text(text):
    return str(text).upper().strip().replace(", ", ",").replace("Á", "A").replace("É", "E").replace("Í", "I").replace("Ó", "O").replace("Ú", "U")

# Preparar datasetF
df_datasetF["NOMBRE_NORM"] = df_datasetF["INVESTIGADOR"].apply(normalize_text)
df_datasetF["UBICACION_NORM"] = (df_datasetF["ENTIDAD FEDERATIVA"] + ", " + df_datasetF["PAÍS"]).apply(normalize_text)

# Preparar INVENTOR6724
df_inventor["NOMBRE_INVENTOR_NORM"] = (
    df_inventor["inventor_last_name"].str.strip() + "," + 
    df_inventor["inventor_first_name"].str.strip()
).apply(normalize_text)
df_inventor["UBICACION_INVENTOR_NORM"] = (df_inventor["inventor_city"] + ", " + df_inventor["inventor_country"]).apply(normalize_text)

# Preparar Match_2_Limpio
df_match["NOMBRE_INVESTIGADOR_NORM"] = df_match["NOMBRE DEL INVESTIGADOR"].apply(normalize_text)
df_match["NOMBRE_PATENTE_NORM"] = df_match["NOMBRE DEL INVESTIGADOR PATENTE"].apply(normalize_text)

# Función para validar identidad
def validar_identidad(row):
    # Buscar en datasetF
    dataF = df_datasetF[df_datasetF["NOMBRE_NORM"] == row["NOMBRE_INVESTIGADOR_NORM"]]
    # Buscar en INVENTOR6724
    inventor = df_inventor[df_inventor["NOMBRE_INVENTOR_NORM"] == row["NOMBRE_PATENTE_NORM"]]
    
    es_misma_persona = False
    detalles = {"coincidencias": []}
    ubicacion_dataF = ""
    ubicacion_inventor = ""
    patent_year = ""
    periodos_validos = set()
    
    # Coincidencia exacta de nombres
    if not dataF.empty and not inventor.empty:
        es_misma_persona = True  # Coincidencia exacta de nombres
        
        # Coincidencia de ubicación (entidad federativa/país vs ciudad/país)
        ubicacion_dataF = dataF.iloc[0]["UBICACION_NORM"]
        ubicacion_inventor = inventor.iloc[0]["UBICACION_INVENTOR_NORM"]
        if ubicacion_dataF in ubicacion_inventor or ubicacion_inventor in ubicacion_dataF:
            detalles["coincidencias"].append("Ubicación")
        
        # Se obtiene el año de la patente
        patent_year = pd.to_datetime(inventor.iloc[0]["patent_date"]).year

        # Iterar sobre cada registro de datasetF para comparar la vigencia
        for idx, fila in dataF.iterrows():
            inicio = pd.to_datetime(fila["FECHA DE INICIO DE VIGENCIA"]).year
            fin = pd.to_datetime(fila["FECHA DE FIN DE VIGENCIA"]).year
            if inicio <= patent_year <= fin:
                periodos_validos.add(f"{inicio}-{fin}")

        if periodos_validos:
            detalles["coincidencias"].append(f"Fecha [patente {patent_year}] válida en: {', '.join(periodos_validos)}")
    
    # Fuzzy matching para nombres no exactos
    else:
        similarity = fuzz.token_set_ratio(row["NOMBRE_INVESTIGADOR_NORM"], row["NOMBRE_PATENTE_NORM"])
        if similarity > 90:
            es_misma_persona = True
            detalles["coincidencias"].append(f"Similitud Nombre ({similarity}%)")
    
    return pd.Series([
        es_misma_persona, 
        ", ".join(set(detalles["coincidencias"])) if detalles["coincidencias"] else "Sin coincidencias", 
        ubicacion_dataF, 
        ubicacion_inventor, 
        patent_year, 
        ", ".join(periodos_validos)
    ])

# Aplicar validación
df_match[["ES_MISMA_PERSONA", "DETALLES", "UBICACION_DATAF", "UBICACION_INVENTOR", "AÑO_PATENTE", "PERIODOS_VALIDOS"]] = df_match.apply(validar_identidad, axis=1)

# Resultados
print(df_match[[
    "NOMBRE DEL INVESTIGADOR", 
    "NOMBRE DEL INVESTIGADOR PATENTE", 
    "ES_MISMA_PERSONA", 
    "DETALLES", 
    "UBICACION_DATAF", 
    "UBICACION_INVENTOR", 
    "AÑO_PATENTE",
    "PERIODOS_VALIDOS"
]])

df_match.drop(['NOMBRE_INVESTIGADOR_NORM', 'NOMBRE_PATENTE_NORM'], axis=1, inplace=True)

# Guardar resultados
df_match.to_csv("Match_2_Validos.csv", index=False, encoding='utf-8-sig')

C:\Users\Adrian\AppData\Local\Temp\ipykernel_11964\2230064633.py:58: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fin = pd.to_datetime(fila["FECHA DE FIN DE VIGENCIA"]).year
C:\Users\Adrian\AppData\Local\Temp\ipykernel_11964\2230064633.py:57: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  inicio = pd.to_datetime(fila["FECHA DE INICIO DE VIGENCIA"]).year


               NOMBRE DEL INVESTIGADOR    NOMBRE DEL INVESTIGADOR PATENTE  \
0    ABSALON CONSTANTINO,ANGEL EDUARDO  ABSALON CONSTANTINO,ANGEL EDUARDO   
1            ABURTO ANELL,JORGE ARTURO          ABURTO ANELL,JORGE ARTURO   
2             ACEVES DIEZ,ANGEL EMILIO           ACEVES DIEZ,ANGEL EMILIO   
3                RIVAS ACEVES,SALVADOR                  ACEVES,SALVADOR M   
4    ACOSTA GONZALEZ,GUILLERMO ENRIQUE  ACOSTA GONZALEZ,GUILLERMO ENRIQUE   
..                                 ...                                ...   
971            ZAVALA OLIVARES,GERARDO            ZAVALA OLIVARES,GERARDO   
972                 ZAVALA YOE,RICARDO                     ZAVALA,RICARDO   
973            ZENTENO RAMIREZ,ANTONIO            ZENTENO RAMIREZ,ANTONIO   
974       ZEPEDA PARTIDA,TRINO ARMANDO       ZEPEDA PARTIDA,TRINO ARMANDO   
975                 ZOLOTUKHIN,MIKHAIL                 ZOLOTUKHIN,MIKHAIL   

     ES_MISMA_PERSONA                                   DETALLES  \
0      